In [ ]:
#!pip install -q timesketch_api_client

In [ ]:
pip install ../api_client/python

In [ ]:
from timesketch_api_client import search
from picatrix.lib import state as state_lib

import altair as alt
import numpy as np
import pandas as pd
from timesketch_api_client import config
from timesketch_import_client import helper
from timesketch_import_client import importer
from timesketch_api_client import search




# Create the sketch and import some data

In [ ]:
ts_client = config.get_client(confirm_choices=False)

In [ ]:
ts_client.create_sketch(name="test")

In [ ]:
sketch = ts_client.get_sketch(1)

In [ ]:
import_helper = helper.ImportHelper() 

with importer.ImportStreamer() as streamer:
  streamer.set_sketch(sketch)
  streamer.set_config_helper(import_helper) 

  streamer.set_timeline_name('sigma_events.jsonl')

  streamer.add_file('../test_tools/test_events/sigma_events.jsonl')

# Multiple Rules


In [ ]:
rules = ts_client.list_sigma_rules()

In [ ]:
for rule in rules:
    rule.__dict__

In [ ]:
rule1 = rules[0]

In [ ]:
rule1.id

In [ ]:
rule1.es_query

# Single rule

In [ ]:
# Single Rule tests
single_rule = ts_client.get_sigma_rule(rule_uuid='5266a592-b793-11ea-b3de-0242ac130004')

In [ ]:
single_rule.__dict__

In [ ]:
single_rule.id

In [ ]:
single_rule.rule_uuid


In [ ]:
single_rule.es_query

# Text Sigma examples

In [ ]:
MOCK_SIGMA_RULE = """
title: Suspicious Installation of Zenmap
id: 5266a592-b793-11ea-b3de-0242ac130004
description: Detects suspicious installation of Zenmap
references:
    - https://rmusser.net/docs/ATT&CK-Stuff/ATT&CK/Discovery.html
author: Alexander Jaeger
date: 2020/06/26
modified: 2021/01/01
logsource:
    product: linux
    service: shell
detection:
    keywords:
        # Generic suspicious commands
        - '*apt-get install zmap*'
    condition: keywords
falsepositives:
    - Unknown
level: high
"""

In [ ]:
rule_by_text = ts_client.get_sigma_rule_by_text(MOCK_SIGMA_RULE)

In [ ]:
rule_by_text.es_query

In [ ]:
rule_by_text.references

# Analyzer

In [ ]:
timelines = sketch.list_timelines()


In [ ]:
timelines[0].name #sigma_events.jsonl

In [ ]:
result = sketch.run_analyzer(timeline_name='sigma_events.jsonl', analyzer_name='sigma', ignore_previous=True)

In [ ]:
sketch.get_analyzer_status()
"""This should show something like:

[{'index': <timesketch_api_client.index.SearchIndex at 0x126a09820>,
  'timeline_id': 1,
  'session_id': 1,
  'analyzer': 'sigma',
  'results': 'Applied 1 tags\n* lnx_susp_zenmap.yml: 1\nProblematic rules:',
  'status': 'DONE'}]
  """